In [32]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint
import time

api_key = 'awPgd5YOKYVv0DCC8BERPRPeKJhXmGw4'

docs = []

def get_articles(query, api_key, page=0, filter_params='&facet=true&begin_date=20170101&end_date=20240901'):
    # Adding filter query (fq) for more precise control
    fq = 'fq=body:("Banana Republic") AND body:("clothing") AND NOT body:("Trump") AND NOT body:("Israel")'
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&page={page}&api-key={api_key}&{fq}{filter_params}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()

for i in range(10): # For 100 articles
    print(i)
    articles = get_articles("Banana Republic", api_key, page = i)
    x = articles['response']['docs']
    docs.extend(x)
    time.sleep(12)


    
text = []
for doc in docs:
    text.append(doc['headline']['main'] + '. ' +  doc['abstract'])
text



0


KeyboardInterrupt: 

In [14]:
def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('Files_Directory/Twitter/AFINN-111.txt')
# Scores_Table = read_scores('AFINN-111.txt')

In [18]:
scores = []

for txt in text:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)
    


In [20]:
!pip install azure-ai-textanalytics

import requests
import azure
import time

key = 'ce14ce8f012d462fbbedd4c70565b678' #"<paste-your-text-analytics-key-here>"
endpoint = 'https://fischba-webdataproject-fall2024.openai.azure.com/' #"<paste-your-text-analytics-endpoint-here>"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client


def sentiment_analysis_example(client, txt):
    documents = [txt]
    response = client.analyze_sentiment(documents=documents)[0]
    print(response)
    responses_all.append(response)

def key_phrase_extraction_example(client, txt):

    try:
        documents = [txt]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            return response.key_phrases
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))

azure_keyphrases = []
azure_sentiments = []
responses_all =[]
txt_number = 0
requests_per_second_limit = 100
time_counter = time.time()  # Start time of processing the request
for txt in text:
    txt_number+=1
    print(txt_number)
    client = authenticate_client()
 # Detect key phrases.
    key_phrases = key_phrase_extraction_example(client,txt)
    azure_keyphrases.append(key_phrases)
 # Detect Sentiments
    sentiments = sentiment_analysis_example(client, txt)

    end_time = time.time()  # End time of processing the request
    processing_time = end_time - time_counter

    requests_in_this_second = txt_number % requests_per_second_limit

    # If we reach the limit of requests per second, add a delay to maintain the limit
    if requests_in_this_second == 0:
        time_taken_for_batch = end_time - time_counter
        if time_taken_for_batch < 1:
            time.sleep(1 - time_taken_for_batch)
        time_counter = time.time()
        
final_sentiments=[]
for i in responses_all:
    x = i['sentiment']
    final_sentiments.append(x)
    
final_sentiments

1
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.09, neutral=0.91, negative=0.0), 'sentences': [SentenceSentiment(text=Banana Republic Wants to Outfit Your Home, Too. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.1, neutral=0.9, negative=0.0), length=48, offset=0, mined_opinions=[]), SentenceSentiment(text=Responding to a volatile market for work clothes, the retailer hopes that selling home goods will help stabilize its business., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.92, negative=0.0), length=126, offset=48, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
2
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.9, negative=0.08), 'sentences': [SentenceSentiment(text=Gap Sees Its Post-Pandemic Future Outside of Malls

{'id': '0', 'sentiment': 'positive', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.53, neutral=0.47, negative=0.0), 'sentences': [SentenceSentiment(text=A Week of Pop-Ups, Sales and Other Shopping Events. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.0), length=52, offset=0, mined_opinions=[]), SentenceSentiment(text=In the Hamptons, summer pop-ups; , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.08, neutral=0.91, negative=0.0), length=33, offset=52, mined_opinions=[]), SentenceSentiment(text=in the city, a fresh start at Dover Street Market, a flurry of end-of-season sales and more., sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=0.53, neutral=0.47, negative=0.0), length=92, offset=85, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
11
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.24, negative=0.75), 'sentences': [SentenceSentiment(text=In Hong Kong, Jimmy Lai’s Media Company Moves to Close Down. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.9, negative=0.09), length=61, offset=0, mined_opinions=[]), SentenceSentiment(text=Next Digital, which has published criticism of China for decades, said a crackdown had left it with no way to operate. , sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.24, negative=0.75), length=119, offset=61, mined_opinions=[]), SentenceSentiment(text=Its main newspaper, Apple Daily, closed in June., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.94, negative=0.05), length=48, offset=180, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
21
{'id': '0', 'senti

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.03, neutral=0.82, negative=0.15), 'sentences': [SentenceSentiment(text=The Biggest Influencers of the Pandemic May Not Be Who You Assume. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.84, negative=0.16), length=67, offset=0, mined_opinions=[]), SentenceSentiment(text=Forget Instagram. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.63, negative=0.37), length=18, offset=67, mined_opinions=[]), SentenceSentiment(text=Cherchez the costume designers. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.97, negative=0.0), length=32, offset=85, mined_opinions=[]), SentenceSentiment(text=It’s time they got their due., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.07, neutral=0.85, negative=0.08), length=29, offset=117

['neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral']

In [31]:
print(final_sentiments.count('positive'))
print(final_sentiments.count('negative'))

3
7
